In [1]:
# [1] 환경 준비
!apt-get update
!apt-get install -y openjdk-8-jdk fonts-nanum > /dev/null
!pip install --upgrade pip setuptools wheel
!pip install JPype1==1.4.1 konlpy scikit-learn --quiet

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:


# [2] 예시 가사 데이터 만들기 (10개 샘플)
lyrics = [
    "행복했던 기억이 가득해",        # 긍정(1)
    "너와의 이별이 너무 아파",        # 부정(0)
    "사랑은 다시 찾아올 거야",       # 긍정(1)
    "외로운 밤 혼자 울고 있어",      # 부정(0)
    "햇살처럼 눈부신 하루야",        # 긍정(1)
    "추억만이 남아 나를 괴롭혀",     # 부정(0)
    "새로운 시작이 설레여",         # 긍정(1)
    "상처뿐인 기억들이 떠올라",      # 부정(0)
    "희망을 안고 달려가",            # 긍정(1)
    "어둠 속에 방황하는 나"          # 부정(0)
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1=긍정, 0=부정

# [3] 형태소 분석(명사+형태소)
from konlpy.tag import Okt

okt = Okt()
def tokenize(text):
    # 형태소 분해(명사+형태소 결합)
    return ' '.join(okt.morphs(text))

tokens = [tokenize(lyric) for lyric in lyrics]
print("형태소 분석 결과 예시:", tokens[:2])


형태소 분석 결과 예시: ['행복했던 기억 이 가득해', '너 와의 이별 이 너무 아파']


In [3]:

# [4] 벡터화(CountVectorizer) + 감성 분류(나이브 베이즈)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(tokens)
X_train, X_test, y_train, y_test = train_test_split(X_vec, labels, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("\n[감성분석 결과]\n")
print(classification_report(y_test, pred, target_names=["부정", "긍정"]))

# [5] 새로운 가사 감성 예측
test_lyrics = [
    "이별의 슬픔이 내 마음을 적셔",
    "희망찬 내일이 기다리고 있어"
]
test_tokens = [tokenize(l) for l in test_lyrics]
test_vec = vectorizer.transform(test_tokens)
preds = clf.predict(test_vec)
for lyric, label in zip(test_lyrics, preds):
    print(f'"{lyric}" => {"긍정" if label==1 else "부정"}')



[감성분석 결과]

              precision    recall  f1-score   support

          부정       0.50      1.00      0.67         1
          긍정       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

"이별의 슬픔이 내 마음을 적셔" => 부정
"희망찬 내일이 기다리고 있어" => 부정


/Users/kwangsiklee/miniforge3/envs/hf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/kwangsiklee/miniforge3/envs/hf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/kwangsiklee/miniforge3/envs/hf_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 